In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import retro

import numpy as np
from PIL import Image
import cv2 #opencv
import io
import time
import pandas as pd
import numpy as np
from IPython.display import clear_output
from random import randint
import random
import os

#from selenium import webdriver
#from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.common.keys import Keys

#keras imports
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from collections import deque
import random
import pickle
from io import BytesIO
import base64
import json

Using TensorFlow backend.


In [3]:
#path variables
#game_url = "chrome://dino"
#chrome_driver_path = "/usr/local/bin/chromedriver"
#chrome_driver_path = "./chromedriver"
loss_file_path = "./objects/loss_df.csv"
actions_file_path = "./objects/actions_df.csv"
q_value_file_path = "./objects/q_values.csv"
scores_file_path = "./objects/scores_df.csv"

#scripts
#create id for canvas for faster selection from DOM
#init_script = "document.getElementsByClassName('runner-canvas')[0].id = 'runner-canvas'"

#get image from canvas
#getbase64Script = "canvasRunner = document.getElementById('runner-canvas'); \
#return canvasRunner.toDataURL().substring(22)"

In [4]:
class Game:
    def __init__(self,env_name):
        self.env = retro.make(game=env_name)
    def restart(self):
        obs = self.env.reset()
        return obs
    def render(self):
        self.env.render()
        
    def Run(self):
        act = [0,0,0,0,0,0,0,1,0]
        return act
    def Stop(self):
        act = [0,0,0,0,0,0,1,1,0]
        return act
    def Jump(self):
        act = [0,0,0,0,0,0,0,1,1]
        return act

    def get_score(self, act):
        obs, rew, done, info = self.env.step(act)
        
        return obs, rew, done, info
    
    def get_action_sample(self):
        act = self.env.action_space.sample()
        return act
    
    def end(self):
        self.env.close()

In [5]:
class clown_agent:
    def __init__(self,game): #takes game as input for taking actions
        self._game = game
        
    def Run(self):
        self._game.Run()
    def Jump(self):
        self._game.Jump()
    def Stop(self):
        self._game.Stop()

In [6]:
class Game_state_:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = show_img() #display the processed image on screen using openCV, implemented using python coroutine 
        self._display.__next__() # initiliaze the display coroutine 
    def get_state(self,actions):
        
        image, reward, is_over, info = self._game.get_score(actions)
        
        reward_1 = 0.1 
        
        image_ap = process_img(image)
        
        actions_df.loc[len(actions_df)] = actions[1] # storing actions in a dataframe
        
        self._display.send(image_ap) #display the image on screen
        
        if is_over:
            score = info['score']
            reward_1 = -1
            scores_df.loc[len(loss_df)] = score # log the score when game is over
            self._game.restart()
        
        return image_ap, reward_1, is_over #return the Experience tuple

In [7]:
def save_obj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f: #dump files into objects folder
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def process_img(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #RGB to Grey Scale
    image = image[35:135, 0:120] #Crop Region of Interest(ROI)
    image = cv2.resize(image, (80,80))
    return  image

def show_img(graphs = False):
    """
    Show images in new window
    """
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)        
        imS = cv2.resize(screen, (600, 600)) 
        cv2.imshow(window_title, screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

In [8]:
#Intialize log structures from file if exists else create new
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
scores_df = pd.read_csv(scores_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns = ['scores'])
actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])
q_values_df =pd.read_csv(actions_file_path) if os.path.isfile(q_value_file_path) else pd.DataFrame(columns = ['qvalues'])

In [9]:
#game parameters
ACTIONS = 2 # possible actions: jump, do nothing
Key_num = 9
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 100. # timesteps to observe before training
EXPLORE = 100000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 16 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4
img_rows , img_cols = 80,80
img_channels = 4 #We stack 4 frames

In [10]:
# training variables saved as checkpoints to filesystem to resume training from the same step
def init_cache():
    """initial variable caching, done only once"""
    save_obj(INITIAL_EPSILON,"epsilon")
    t = 0
    save_obj(t,"time")
    D = deque()
    save_obj(D,"D")

In [11]:
'''Call only once to init file structure
'''
#init_cache()

'Call only once to init file structure\n'

In [12]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Conv2D(32, (8, 8), padding='same',strides=(4, 4),input_shape=(img_cols,img_rows,img_channels)))  #80*80*4
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4),strides=(2, 2),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3),strides=(1, 1),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    
    #create model file if not present
    if not os.path.isfile(loss_file_path):
        model.save_weights('model.h5')
    print("We finish building the model")
    return model

In [13]:
''' 
main training module
Parameters:
* model => Keras Model to be trained
* game_state => Game State module with access to game environment and dino
* observe => flag to indicate wherther the model is to be trained(weight updates), else just play
'''
def trainNetwork(model,game_state,observe=False):
    last_time = time.time()
    # store the previous observations in replay memory
    D = load_obj("D") #load from file system
    # get the first state by doing nothing
    do_nothing = np.zeros(Key_num)
    #do_nothing[0] =1 #0 => do nothing,
                     #1=> jump
    
    x_t, r_0, terminal = game_state.get_state(do_nothing) # get next step after performing the action
    

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) # stack 4 images to create placeholder input
    

    
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*20*40*4
    
    initial_state = s_t 

    if observe :
        OBSERVE = 9999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = load_obj("epsilon") 
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)

    t = load_obj("time") # resume from the previous time step stored in file system
    while (True): #endless running
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 #reward at 4
        a_t = np.zeros(Key_num) # action at t
        #run, jump
        
        
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0: #parameter to skip frames for actions
            if  random.random() <= epsilon: #randomly explore an action
                print("----------Random Action----------")
                #action_index = random.randrange(ACTIONS)
                a_t = random.choice([[0,0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1,1]])
                
            else: # predict the output
                q = model.predict(s_t)       # input a stack of 4 images, get the prediction
                print(q)
                max_Q = np.argmax(q)         # chosing index with maximum q value
                action_index = max_Q
                a_t_all = [[0,0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1,1]]
                a_t = a_t_all[action_index]       # run, jump
                
        #We reduced the epsilon (exploration parameter) gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE 

        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        
            
        print('fps: {0}'.format(1 / (time.time()-last_time))) # helpful for measuring frame rate
        last_time = time.time()
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x20x40x1
        
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # append the new image to input stack and remove the first one
        
        
        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 20, 40, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]    # 4D stack of images
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]   #reward at state_t due to action_t
                state_t1 = minibatch[i][3]   #next state
                terminal = minibatch[i][4]   #wheather the agent died or survided due the action
                

                inputs[i:i + 1] = state_t    

                targets[i] = model.predict(state_t)  # predicted q values
                 
                Q_sa = model.predict(state_t1)      #predict q values for next step
                
                if terminal:
                    targets[i, action_t] = reward_t # if terminated, only equals reward
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
            q_values_df.loc[len(q_values_df)] = np.max(Q_sa)
        s_t = initial_state if terminal else s_t1 #reset game to initial frame if terminate
        t = t + 1
        
        # save progress every 1000 iterations
        if t % 1000 == 0:
            print("Now we save model")
            #game_state._game.pause() #pause game while saving to filesystem
            model.save_weights("model.h5", overwrite=True)
            save_obj(D,"D") #saving episodes
            save_obj(t,"time") #caching time steps
            save_obj(epsilon,"epsilon") #cache epsilon to avoid repeated randomness in actions
            loss_df.to_csv("./objects/loss_df.csv",index=False)
            scores_df.to_csv("./objects/scores_df.csv",index=False)
            actions_df.to_csv("./objects/actions_df.csv",index=False)
            q_values_df.to_csv(q_value_file_path,index=False)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)
            clear_output()
            #game_state._game.resume()
        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state,             "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t,             "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")

In [14]:
'''import retro

def main():
    
    env_name = 'CircusCharlie-Nes'
    game = Game(env_name)
    
    game.restart()
    
    while True:
        obs, rew, done, info = game.get_score(game.Jump())
        game.render()
        if info['lives']<3:
            game.restart()
    game.end()'''

"import retro\n\ndef main():\n    \n    env_name = 'CircusCharlie-Nes'\n    game = Game(env_name)\n    \n    game.restart()\n    \n    while True:\n        obs, rew, done, info = game.get_score(game.Jump())\n        game.render()\n        if info['lives']<3:\n            game.restart()\n    game.end()"

In [15]:
'''import retro

#main function
def main(observe=True):
    env_name = 'CircusCharlie-Nes'
    game = Game(env_name)
    game.restart()
    
    clown = clown_agent(game)
    game_state = Game_state_(clown,game)  
    model = buildmodel()
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.restart()
'''

"import retro\n\n#main function\ndef main(observe=True):\n    env_name = 'CircusCharlie-Nes'\n    game = Game(env_name)\n    game.restart()\n    \n    clown = clown_agent(game)\n    game_state = Game_state_(clown,game)  \n    model = buildmodel()\n    try:\n        trainNetwork(model,game_state,observe=observe)\n    except StopIteration:\n        game.restart()\n"

In [16]:
import retro

#main function
def main(observe=False):
    env_name = 'CircusCharlie-Nes'
    game = Game(env_name)
    game.restart()
    
    clown = clown_agent(game)
    game_state = Game_state_(clown,game)
    model = buildmodel()
    
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.end()

In [ ]:
if __name__ == "__main__":
    main()

TIMESTEP 1000 / STATE explore / EPSILON 0.09910189899999863 / ACTION 0 / REWARD 0.1 / Q_MAX  10.10932 / Loss  0.006771107669919729
[[6.424925 4.652093]]
fps: 9.292714809218484
TIMESTEP 1001 / STATE explore / EPSILON 0.09910089999999863 / ACTION 0 / REWARD 0.1 / Q_MAX  6.990325 / Loss  0.011343286372721195
[[6.466568  4.6852055]]
fps: 22.43195225132234
TIMESTEP 1002 / STATE explore / EPSILON 0.09909990099999863 / ACTION 0 / REWARD 0.1 / Q_MAX  6.993907 / Loss  0.410549134016037
[[6.5531745 4.723583 ]]
fps: 21.104584404671453
TIMESTEP 1003 / STATE explore / EPSILON 0.09909890199999863 / ACTION 0 / REWARD 0.1 / Q_MAX  9.743923 / Loss  0.040397919714450836
[[6.666066 4.766095]]
fps: 22.369263425118532
TIMESTEP 1004 / STATE explore / EPSILON 0.09909790299999863 / ACTION 0 / REWARD 0.1 / Q_MAX  7.142787 / Loss  0.014174476265907288
[[6.772915  4.8020487]]
fps: 23.124656794098513
TIMESTEP 1005 / STATE explore / EPSILON 0.09909690399999863 / ACTION 0 / REWARD 0.1 / Q_MAX  9.8476 / Loss  0.0126

TIMESTEP 1047 / STATE explore / EPSILON 0.09905494599999856 / ACTION 0 / REWARD 0.1 / Q_MAX  11.52252 / Loss  1.0225578546524048
[[10.3487625  5.9441795]]
fps: 20.963868188111338
TIMESTEP 1048 / STATE explore / EPSILON 0.09905394699999856 / ACTION 0 / REWARD 0.1 / Q_MAX  12.251524 / Loss  0.03145337849855423
[[10.621198  6.006539]]
fps: 22.256498967912425
TIMESTEP 1049 / STATE explore / EPSILON 0.09905294799999856 / ACTION 0 / REWARD 0.1 / Q_MAX  12.302928 / Loss  0.18767209351062775
[[10.783746   6.0468335]]
fps: 22.784233627395594
TIMESTEP 1050 / STATE explore / EPSILON 0.09905194899999856 / ACTION 0 / REWARD 0.1 / Q_MAX  12.455577 / Loss  0.020439524203538895
[[10.831314  6.05502 ]]
fps: 21.735298385257966
TIMESTEP 1051 / STATE explore / EPSILON 0.09905094999999856 / ACTION 0 / REWARD 0.1 / Q_MAX  10.831314 / Loss  0.03516937419772148
[[10.792735  6.038639]]
fps: 23.203589269809306
TIMESTEP 1052 / STATE explore / EPSILON 0.09904995099999855 / ACTION 0 / REWARD 0.1 / Q_MAX  14.587171

TIMESTEP 1095 / STATE explore / EPSILON 0.09900699399999849 / ACTION 0 / REWARD 0.1 / Q_MAX  9.6705885 / Loss  0.015144722536206245
[[12.614359   6.2091885]]
fps: 20.675753348351826
TIMESTEP 1096 / STATE explore / EPSILON 0.09900599499999849 / ACTION 0 / REWARD 0.1 / Q_MAX  11.240154 / Loss  0.013735668733716011
[[13.201887  8.889953]]
fps: 21.584298226654727
TIMESTEP 1097 / STATE explore / EPSILON 0.09900499599999849 / ACTION 0 / REWARD 0.1 / Q_MAX  11.365808 / Loss  0.01425977423787117
[[13.812033   7.7463694]]
fps: 23.22453183313215
TIMESTEP 1098 / STATE explore / EPSILON 0.09900399699999848 / ACTION 0 / REWARD 0.1 / Q_MAX  11.718529 / Loss  0.04133898392319679
[[13.951435  8.411472]]
fps: 22.523139049091945
TIMESTEP 1099 / STATE explore / EPSILON 0.09900299799999848 / ACTION 0 / REWARD 0.1 / Q_MAX  11.720226 / Loss  0.02272012270987034
[[14.324769  8.049128]]
fps: 22.49149524892216
TIMESTEP 1100 / STATE explore / EPSILON 0.09900199899999848 / ACTION 0 / REWARD 0.1 / Q_MAX  3.752812

TIMESTEP 1141 / STATE explore / EPSILON 0.09896103999999842 / ACTION 0 / REWARD 0.1 / Q_MAX  18.309319 / Loss  0.16733939945697784
[[18.988712 17.35681 ]]
fps: 21.115315297173755
TIMESTEP 1142 / STATE explore / EPSILON 0.09896004099999842 / ACTION 0 / REWARD 0.1 / Q_MAX  17.526789 / Loss  0.12363247573375702
[[17.941515 15.784795]]
fps: 21.53542510641138
TIMESTEP 1143 / STATE explore / EPSILON 0.09895904199999841 / ACTION 0 / REWARD 0.1 / Q_MAX  20.070139 / Loss  4.139331817626953
[[17.92264  17.136942]]
fps: 22.58450537379655
TIMESTEP 1144 / STATE explore / EPSILON 0.09895804299999841 / ACTION 0 / REWARD 0.1 / Q_MAX  17.47498 / Loss  0.09263432770967484
[[17.580507 15.438919]]
fps: 22.091096785067204
TIMESTEP 1145 / STATE explore / EPSILON 0.09895704399999841 / ACTION 0 / REWARD 0.1 / Q_MAX  19.30666 / Loss  0.07710748165845871
----------Random Action----------
fps: 21.645004309076928
TIMESTEP 1146 / STATE explore / EPSILON 0.09895604499999841 / ACTION 0 / REWARD 0.1 / Q_MAX  8.724556

TIMESTEP 1190 / STATE explore / EPSILON 0.09891208899999834 / ACTION 0 / REWARD 0.1 / Q_MAX  19.688833 / Loss  0.20438224077224731
[[16.241943  9.768712]]
fps: 21.856944835277073
TIMESTEP 1191 / STATE explore / EPSILON 0.09891108999999834 / ACTION 0 / REWARD 0.1 / Q_MAX  15.561628 / Loss  0.04917873442173004
[[16.87938   9.802779]]
fps: 23.21527647091382
TIMESTEP 1192 / STATE explore / EPSILON 0.09891009099999834 / ACTION 0 / REWARD 0.1 / Q_MAX  15.064734 / Loss  0.06419488787651062
[[16.715767 10.021717]]
fps: 22.42307795116892
TIMESTEP 1193 / STATE explore / EPSILON 0.09890909199999834 / ACTION 0 / REWARD 0.1 / Q_MAX  8.292076 / Loss  0.056783098727464676
[[17.66106  10.113267]]
fps: 21.60008239777526
TIMESTEP 1194 / STATE explore / EPSILON 0.09890809299999834 / ACTION 0 / REWARD 0.1 / Q_MAX  19.288643 / Loss  0.044342659413814545
[[18.076593 10.80931 ]]
fps: 21.41130827190493
TIMESTEP 1195 / STATE explore / EPSILON 0.09890709399999834 / ACTION 0 / REWARD 0.1 / Q_MAX  16.618732 / Los

...

TIMESTEP 1715 / STATE explore / EPSILON 0.09838761399999754 / ACTION 1 / REWARD 0.1 / Q_MAX  34.155148 / Loss  0.03984205424785614
----------Random Action----------
fps: 21.760332036316473
TIMESTEP 1716 / STATE explore / EPSILON 0.09838661499999754 / ACTION 0 / REWARD 0.1 / Q_MAX  25.454533 / Loss  0.13548986613750458
[[24.909918 25.861174]]
fps: 22.318318957489716
TIMESTEP 1717 / STATE explore / EPSILON 0.09838561599999754 / ACTION 1 / REWARD 0.1 / Q_MAX  23.974754 / Loss  0.1645258665084839
[[25.18718  26.040033]]
fps: 24.1679765828474
TIMESTEP 1718 / STATE explore / EPSILON 0.09838461699999754 / ACTION 1 / REWARD 0.1 / Q_MAX  27.11513 / Loss  0.1382313370704651
[[25.825546 26.363169]]
fps: 22.689574587787252
TIMESTEP 1719 / STATE explore / EPSILON 0.09838361799999754 / ACTION 1 / REWARD 0.1 / Q_MAX  26.27709 / Loss  0.040375228971242905
[[26.38634  26.615696]]
fps: 22.843051183460958
TIMESTEP 1720 / STATE explore / EPSILON 0.09838261899999753 / ACTION 1 / REWARD 0.1 / Q_MAX  26.1979

TIMESTEP 1764 / STATE explore / EPSILON 0.09833866299999747 / ACTION 0 / REWARD 0.1 / Q_MAX  29.24856 / Loss  0.05508209019899368
[[10.535911 11.236855]]
fps: 21.525257242564983
TIMESTEP 1765 / STATE explore / EPSILON 0.09833766399999747 / ACTION 1 / REWARD 0.1 / Q_MAX  28.54504 / Loss  0.09493443369865417
[[3.8209145 5.8258986]]
fps: 23.598771197407363
TIMESTEP 1766 / STATE explore / EPSILON 0.09833666499999746 / ACTION 1 / REWARD 0.1 / Q_MAX  0.31121472 / Loss  0.031725384294986725
[[0.3109383  0.06607124]]
fps: 23.25737067698774
TIMESTEP 1767 / STATE explore / EPSILON 0.09833566599999746 / ACTION 0 / REWARD 0.1 / Q_MAX  29.120457 / Loss  0.05696910619735718
[[0.31014714 0.06622311]]
fps: 22.565307681547715
TIMESTEP 1768 / STATE explore / EPSILON 0.09833466699999746 / ACTION 0 / REWARD 0.1 / Q_MAX  26.380674 / Loss  0.18885670602321625
----------Random Action----------
fps: 22.875443131865136
TIMESTEP 1769 / STATE explore / EPSILON 0.09833366799999746 / ACTION 0 / REWARD 0.1 / Q_MAX 

TIMESTEP 1813 / STATE explore / EPSILON 0.09828971199999739 / ACTION 1 / REWARD 0.1 / Q_MAX  9.738997 / Loss  0.12399855256080627
[[9.748791 9.778699]]
fps: 20.709544265047153
TIMESTEP 1814 / STATE explore / EPSILON 0.09828871299999739 / ACTION 1 / REWARD 0.1 / Q_MAX  19.159264 / Loss  0.03612019494175911
[[9.812998 9.808339]]
fps: 21.604866666323268
TIMESTEP 1815 / STATE explore / EPSILON 0.09828771399999739 / ACTION 0 / REWARD 0.1 / Q_MAX  18.519346 / Loss  0.1078772246837616
[[9.847522 9.829828]]
fps: 24.85469295360675
TIMESTEP 1816 / STATE explore / EPSILON 0.09828671499999739 / ACTION 0 / REWARD 0.1 / Q_MAX  16.902052 / Loss  0.03698599711060524
----------Random Action----------
fps: 24.524794910626056
TIMESTEP 1817 / STATE explore / EPSILON 0.09828571599999739 / ACTION 0 / REWARD 0.1 / Q_MAX  19.205454 / Loss  0.20132167637348175
[[9.988513 9.920752]]
fps: 24.225907250495283
TIMESTEP 1818 / STATE explore / EPSILON 0.09828471699999738 / ACTION 0 / REWARD 0.1 / Q_MAX  22.799768 / L

TIMESTEP 1861 / STATE explore / EPSILON 0.09824175999999732 / ACTION 0 / REWARD 0.1 / Q_MAX  19.181387 / Loss  0.025259096175432205
[[11.06434 10.83544]]
fps: 22.313450939501628
TIMESTEP 1862 / STATE explore / EPSILON 0.09824076099999732 / ACTION 0 / REWARD 0.1 / Q_MAX  19.56386 / Loss  0.0460938885807991
[[10.932863 10.740593]]
fps: 22.646940670827846
TIMESTEP 1863 / STATE explore / EPSILON 0.09823976199999732 / ACTION 0 / REWARD 0.1 / Q_MAX  19.022081 / Loss  0.02311321161687374
[[10.78292  10.653556]]
fps: 23.287586961229035
TIMESTEP 1864 / STATE explore / EPSILON 0.09823876299999731 / ACTION 0 / REWARD 0.1 / Q_MAX  17.874199 / Loss  0.022726772353053093
[[10.6000805 10.541061 ]]
fps: 23.66187521155365
TIMESTEP 1865 / STATE explore / EPSILON 0.09823776399999731 / ACTION 0 / REWARD 0.1 / Q_MAX  19.986286 / Loss  0.036419764161109924
[[10.472137 10.453817]]
fps: 21.160803386290368
TIMESTEP 1866 / STATE explore / EPSILON 0.09823676499999731 / ACTION 0 / REWARD 0.1 / Q_MAX  19.781532 / 

TIMESTEP 1911 / STATE explore / EPSILON 0.09819180999999724 / ACTION 0 / REWARD 0.1 / Q_MAX  18.620493 / Loss  0.08209587633609772
[[11.572851 11.431343]]
fps: 21.80739754801543
TIMESTEP 1912 / STATE explore / EPSILON 0.09819081099999724 / ACTION 0 / REWARD 0.1 / Q_MAX  18.22755 / Loss  0.02571801096200943
[[11.334989 11.380422]]
fps: 23.81300706844182
TIMESTEP 1913 / STATE explore / EPSILON 0.09818981199999724 / ACTION 1 / REWARD 0.1 / Q_MAX  17.930658 / Loss  2.699784994125366
[[10.84597   11.1757145]]
fps: 22.412294341760045
TIMESTEP 1914 / STATE explore / EPSILON 0.09818881299999724 / ACTION 1 / REWARD 0.1 / Q_MAX  17.419733 / Loss  0.05011027678847313
[[10.560944 11.072905]]
fps: 21.764622856905643
TIMESTEP 1915 / STATE explore / EPSILON 0.09818781399999724 / ACTION 1 / REWARD 0.1 / Q_MAX  17.067902 / Loss  3.026693820953369
----------Random Action----------
fps: 23.698908934756446
TIMESTEP 1916 / STATE explore / EPSILON 0.09818681499999724 / ACTION 0 / REWARD 0.1 / Q_MAX  18.2112